In [ ]:
import os

os.environ['MLFLOW_TRACKING_URI']="https://dagshub.com/atharvpatwardhan/Wine-Quality-Prediction-Pipeline.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME']="atharvpatwardhan"
os.environ['MLFLOW_TRACKING_PASSWORD']=""

In [2]:
import os
%pwd

'd:\\Projects\\Wine-Quality-Prediction-Pipeline\\research'

In [3]:
os.chdir('../')
%pwd

'd:\\Projects\\Wine-Quality-Prediction-Pipeline'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [5]:
from src.wine_quality_pipline.constants import *
from src.wine_quality_pipline.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet

        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            test_data_path=Path(config.test_data_path),
            model_path=Path(config.model_path),
            all_params=params,
            metric_file_name=Path(config.metric_file_name),
            target_column=schema.name,
            mlflow_uri= "https://dagshub.com/atharvpatwardhan/Wine-Quality-Prediction-Pipeline.mlflow"
        )

        return model_evaluation_config


In [7]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    

    def evaluate_metric(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        X_test = test_data.drop(self.config.target_column, axis=1)
        y_test = test_data[[self.config.target_column]]

        mlflow.set_tracking_uri(self.config.mlflow_uri)
        mlflow.set_experiment("wine_quality_prediction")
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(X_test)

            (rmse, mae, r2) = self.evaluate_metric(y_test, predicted_qualities)

            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(data=scores, path = Path(self.config.metric_file_name))

            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(scores)

            if tracking_uri_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElatsticNetWineQualityPredictionModel")

            else:
                mlflow.sklearn.log_model(model, "model")
    

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-03-14 15:27:09,184: INFO: Loaded file: config\config.yaml]
[2025-03-14 15:27:09,186: INFO: Loaded file: params.yaml]
[2025-03-14 15:27:09,188: INFO: Loaded file: schema.yaml]
[2025-03-14 15:27:09,189: INFO: Created directory at: artifacts]
[2025-03-14 15:27:09,190: INFO: Created directory at: artifacts/model_evaluation]


2025/03/14 15:27:10 INFO mlflow.tracking.fluent: Experiment with name 'wine_quality_prediction' does not exist. Creating a new experiment.


[2025-03-14 15:27:11,066: INFO: json file saved at: artifacts\model_evaluation\metrics.json]


2025/03/14 15:27:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElatsticNetWineQualityPredictionModel'.
2025/03/14 15:27:23 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElatsticNetWineQualityPredictionModel, version 1
Created version '1' of model 'ElatsticNetWineQualityPredictionModel'.


🏃 View run big-owl-648 at: https://dagshub.com/atharvpatwardhan/Wine-Quality-Prediction-Pipeline.mlflow/#/experiments/0/runs/08435ddc6ebc45fcbcff9ff10dafafc6
🧪 View experiment at: https://dagshub.com/atharvpatwardhan/Wine-Quality-Prediction-Pipeline.mlflow/#/experiments/0
